### temporary testing for accuracy

In [5]:
#  imports
import os
import numpy as np
import librosa
import numpy as np
from scipy.spatial.distance import euclidean
import matplotlib.pyplot as plt
from tqdm import tqdm

from chromagram import fetch_chromagram
from DTW import dtw_table, dtw, display_dtw_path, similarity_score

#### generate chromagrams for train files from dataset

In [6]:
# fetch file pairs


gen_path = "../dataset3/archive/musicnet/custom_generated_musicnet/"
og_path = "../dataset3/archive/musicnet/musicnet/train_data/"

# where train files chromagrams will be saved, to be loaded and used during inference
chroma_path = "../chromagrams/30sec_sr1000_numpy_files/"

file_pairs = []
no_pairs = []
multiple_pairs = []

og_files = os.listdir(og_path)


for file in tqdm(os.listdir(gen_path)):
    file_num = file.split('_')[0]
    topair = [f for f in og_files if file_num in f]
    if len(topair)==0:
        no_pairs.append(file)
        continue
    elif len(topair)>1:
        multiple_pairs.append([file,topair])
        continue
    topair = topair[0]
    file_pairs.append([topair, file])
file_pairs = np.array(file_pairs)

100%|█████████████████████████████████████████████████████████████████████████████| 330/330 [00:00<00:00, 25417.69it/s]


In [8]:
# save chromagrams

og_files = file_pairs[:,0]
for ogf in tqdm(og_files):
    # saving 30 seconds chromagrams of all train files with sampling rate 10k
    # change accordingly
    chromagram = fetch_chromagram(og_path+ogf, sr=1000, end_time=30)
    np.save(chroma_path+ogf[:-3]+'npy', chromagram)

100%|████████████████████████████████████████████████████████████████████████████████| 320/320 [01:15<00:00,  4.26it/s]


#### inference (computing closest matches for test files)

In [9]:
def method1(chromagram1, chromagram2):
    # dtw from MIR with cost without normalization
    D = dtw_table(chromagram1.T, chromagram2.T)
    dtw_path = dtw(chromagram1.T, chromagram1.T, D)

    score = similarity_score(D, dtw_path)
    
    return score

def method2(chromagram1, chromagram2):
    # librosa dtw with normalized similarity
    return 0

In [18]:
chroma_files = os.listdir(chroma_path)

# each element in final_maps contains [original file, generated midi/test file, closest match file]
final_maps = []
all_scores = []

for og_file, gen_file in tqdm(file_pairs):
    # load test file chromagram
    chromagram1 = fetch_chromagram(gen_path+gen_file, sr=1000, end_time=30)
    scores = []
    for chroma_file in chroma_files:
        # fetch each train file chromagram
        chromagram2 = np.load(chroma_path+chroma_file)
        
        score = method1(chromagram1, chromagram2)
        
        scores.append(score)
        
        
    matched_file = chroma_files[np.argmin(scores)]    # argmin in case of cost, argmax in case of similarity
    
    # each element in final_maps contains [original file, generated midi/test file, closest match file]
    final_maps.append([og_file, gen_file, matched_file])
    
    all_scores.append(scores)
    
    
#     break    # test for one test file print final_maps to check, remove break later

 32%|██████████████████████████                                                      | 104/320 [28:31<57:07, 15.87s/it]C:\Users\srava\anaconda3\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=12
  warnings.warn(
C:\Users\srava\anaconda3\lib\site-packages\librosa\core\pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
100%|██████████████████████████████████████████████████████████████████████████████| 320/320 [1:20:28<00:00, 15.09s/it]


In [19]:
np.save('./all_scores.npy', all_scores)

##### accuracy

In [20]:
accuracy = 0
for i in final_maps:
    ground_truth = i[0][:-3]
    closest_match = i[2][:-3]
    accuracy += ground_truth==closest_match
accuracy = accuracy/len(final_maps)
accuracy*100

58.12500000000001

In [21]:
accuracy

0.58125

In [22]:
final_maps

[['1727.wav', '1727_schubert_op114_2.wav', '1727.npy'],
 ['1728.wav', '1728_schubert_op114_3.wav', '1728.npy'],
 ['1729.wav', '1729_schubert_op114_4.wav', '2482.npy'],
 ['1730.wav', '1730_schubert_op114_5.wav', '1730.npy'],
 ['1733.wav', '1733_sy_sps92.wav', '1733.npy'],
 ['1734.wav', '1734_sy_sps93.wav', '1734.npy'],
 ['1735.wav', '1735_sy_sps94.wav', '1730.npy'],
 ['1739.wav', '1739_sb99m4.wav', '1739.npy'],
 ['1742.wav', '1742_sb163m2.wav', '2523.npy'],
 ['1749.wav', '1749_sy_sps11.wav', '1749.npy'],
 ['1750.wav', '1750_sy_sps12.wav', '1750.npy'],
 ['1751.wav', '1751_sy_sps13.wav', '1751.npy'],
 ['1752.wav', '1752_sy_sps14.wav', '1752.npy'],
 ['1755.wav', '1755_sy_sps52.wav', '2390.npy'],
 ['1756.wav', '1756_sy_sps53.wav', '2463.npy'],
 ['1757.wav', '1757_d958-1.wav', '1807.npy'],
 ['1758.wav', '1758_d958-2.wav', '2167.npy'],
 ['1760.wav', '1760_d958-4.wav', '2576.npy'],
 ['1763.wav', '1763_scbt1421.wav', '2156.npy'],
 ['1764.wav', '1764_scbt1422.wav', '1764.npy'],
 ['1765.wav', '17

In [26]:
np.argmin(all_scores[2])

242

In [31]:
sorted(all_scores[0])

[28.555509326250657,
 31.8876950947138,
 32.35969098499327,
 32.557165722052254,
 32.84283244916622,
 33.41029123297105,
 33.583188535438644,
 33.63853698259308,
 34.035347319583394,
 34.22037788471089,
 34.25268284928414,
 34.62200693806557,
 34.8699067896232,
 34.89590328014814,
 34.900202283783564,
 34.92787119877689,
 35.12421002747521,
 35.13084367202485,
 35.26354754886457,
 35.26992918189083,
 35.42623961152452,
 35.80077062661831,
 35.82009285176173,
 35.94480213610565,
 36.07995762536302,
 36.21277148688017,
 36.40911216395242,
 36.47438472682151,
 36.560503867039316,
 36.586948493315326,
 36.88003616382678,
 36.98850532900542,
 37.036021461710334,
 37.054546031881785,
 37.08244157814589,
 37.59267507819459,
 37.612686946758856,
 37.6507017079924,
 37.69080234661935,
 37.70587195341404,
 37.74768914704892,
 37.820356918162986,
 37.85195607308185,
 37.87261322531544,
 37.92436740375482,
 37.926030366681516,
 38.1045234072389,
 38.26107808706515,
 38.2694463233153,
 38.365249618

### temporary testing for accuracy

In [32]:
#  imports
import os
import numpy as np
import librosa
import numpy as np
from scipy.spatial.distance import euclidean
import matplotlib.pyplot as plt
from tqdm import tqdm

from chromagram import fetch_chromagram
from DTW import dtw_table, dtw, display_dtw_path, similarity_score

#### generate chromagrams for train files from dataset

In [33]:
# fetch file pairs


gen_path = "../dataset3/archive/musicnet/custom_generated_musicnet/"
og_path = "../dataset3/archive/musicnet/musicnet/train_data/"

# where train files chromagrams will be saved, to be loaded and used during inference
chroma_path = "../chromagrams/30sec_sr1000_numpy_files/"

file_pairs = []
no_pairs = []
multiple_pairs = []

og_files = os.listdir(og_path)


for file in tqdm(os.listdir(gen_path)):
    file_num = file.split('_')[0]
    topair = [f for f in og_files if file_num in f]
    if len(topair)==0:
        no_pairs.append(file)
        continue
    elif len(topair)>1:
        multiple_pairs.append([file,topair])
        continue
    topair = topair[0]
    file_pairs.append([topair, file])
file_pairs = np.array(file_pairs)

100%|█████████████████████████████████████████████████████████████████████████████| 330/330 [00:00<00:00, 28748.40it/s]


In [8]:
# save chromagrams

og_files = file_pairs[:,0]
for ogf in tqdm(og_files):
    # saving 30 seconds chromagrams of all train files with sampling rate 10k
    # change accordingly
    chromagram = fetch_chromagram(og_path+ogf, sr=1000, end_time=30)
    np.save(chroma_path+ogf[:-3]+'npy', chromagram)

100%|████████████████████████████████████████████████████████████████████████████████| 320/320 [01:15<00:00,  4.26it/s]


#### inference (computing closest matches for test files)

In [34]:
def method1(chromagram1, chromagram2):
    # dtw from MIR with cost without normalization
    D = dtw_table(chromagram1.T, chromagram2.T)
    dtw_path = dtw(chromagram1.T, chromagram1.T, D)

    score = similarity_score(D, dtw_path)
    
    return score

def method2(chromagram1, chromagram2):
    # librosa dtw with normalized similarity
    return 0

In [36]:
chroma_files = os.listdir(chroma_path)

# each element in final_maps contains [original file, generated midi/test file, closest match file]
final_maps = []
all_scores = []

for og_file, gen_file in tqdm(file_pairs):
    # load test file chromagram
    chromagram1 = fetch_chromagram(gen_path+gen_file, sr=1000, end_time=30)
    scores = []
    for chroma_file in tqdm(chroma_files):
        # fetch each train file chromagram
        chromagram2 = np.load(chroma_path+chroma_file)
        
        score = method1(chromagram1, chromagram2)
        
        scores.append(score)
        
        
    matched_file = chroma_files[np.argmin(scores)]    # argmin in case of cost, argmax in case of similarity
    
    # each element in final_maps contains [original file, generated midi/test file, closest match file]
    final_maps.append([og_file, gen_file, matched_file])
    
    all_scores.append(scores)
    
    
    break    # test for one test file print final_maps to check, remove break later

 40%|████████████████████████████████                                                | 128/320 [00:08<00:13, 14.27it/s]


 81%|█████████████████████████████████████████████████████████████████               | 260/320 [00:17<00:04, 14.97it/s]


  0%|                                                                                          | 0/320 [00:22<?, ?it/s]
